<img src="http://www.chad02129.org/wp-content/uploads/2013/12/Landscape-BallsonRack.png" style="width:1000px;height:100px;" />

The purpose of this Notebook is to construct a cool dataset containing data about men matchs of the last seven [Roland Garros](http://www.rolandgarros.com/en_FR/news/index.html).
We will scrap data from *Equipe* newspaper website.

In [1]:
from urllib import request
from bs4 import BeautifulSoup
from pandas import DataFrame

Here is the list of the *Equipe* newspaper urls from where we scrap:

In [2]:
list_tournois = {
    "2016" : "http://www.lequipe.fr/Tennis/SH_RG.html" ,
    "2015" : "http://www.lequipe.fr/Tennis/TennisTableauTournoi5637.html",
    "2014" : "http://www.lequipe.fr/Tennis/TennisTableauTournoi4939.html",
    "2013" : "http://www.lequipe.fr/Tennis/TennisTableauTournoi4456.html",
    "2012" : "http://www.lequipe.fr/Tennis/TennisTableauTournoi3687.html",
    "2011" : "http://www.lequipe.fr/Tennis/TennisTableauTournoi3114.html",
    "2010" :"http://www.lequipe.fr/Tennis/TennisTableauTournoi2672.html"
 }

In [3]:
def is_joueur(tag):
    return tag.has_attr('nomcomplet')

Data will be stored in 5 lists.
Here is the hierarchical structure of the Equipe website table:
- table
    - rounds
        - matchs
            - players
            - scores

In [4]:
winners = []
losers = []
years = []
scores_winners = []
scores_losers = []
rounds = []

for year in list_tournois.keys():
    data = request.urlopen(list_tournois[year]).read()
    soup = BeautifulSoup(data, 'html.parser')
    
    table = soup.find(id = "rateau") #table is a html element containing all the data we want !

    dict_rounds = {}
    for i in [4,5,6]: # I only take the three last rounds. It is possible to take more.
        dict_rounds[i] = table.findAll("div", attrs = {"numtour": str(i)})[0] #we first parse table in rounds

    dict_match = {}
    for i in dict_rounds.keys(): #we parse dict_rounds[i] (which is the i-th rounds) in matchs
            dict_match[i] = dict_rounds[i].findAll("div", attrs = {"class": "joueurs"})

    for i in dict_match.keys(): #dict_match[i] is the matchs list corresponding to the round i
        for k in dict_match[i]: #k is a match
            years += [year]
            rounds += [i]
            for j in k.findAll("a"):
                if "gagne" in j["class"]:
                    winners += [j.findAll(is_joueur)[0]["nomcomplet"] ]
                    scores_winners += [j.findAll(attrs={"class": "score"})[0].text]
                else:
                    losers += [j.findAll(is_joueur)[0]["nomcomplet"] ]
                    scores_losers += [j.findAll(attrs={"class": "score"})[0].text]    

Now we can construct the Data Frame from our list : 

In [5]:
df = DataFrame({"Winners": winners, "Losers": losers, "Years" : years, "Rounds" : rounds,
                "Scores_Winners": scores_winners, "Scores_Losers": scores_losers})
df.head()

,Losers,Rounds,Scores_Losers,Scores_Winners,Winners,Years
0,TSONGA,4,177661,65576,DJOKOVIC,2012
1,DEL POTRO,4,674203,36666,FEDERER,2012
2,MURRAY,4,47332,6666,FERRER,2012
3,ALMAGRO,4,6423,766,NADAL,2012
4,FEDERER,5,453,676,DJOKOVIC,2012


In [6]:
df.to_csv("rolland_garros.csv", index=False)

I used this dataset to construct a interactive graph with [D3](https://d3js.org/). Come see it [here](http://bl.ocks.org/rhuille/8c1c29a182f186d3f17a1f67f90faf96) !